<a href="https://colab.research.google.com/github/medbar/maga_sis/blob/main/2/ED/kaggle_wav2vec2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch 

In [ ]:
!pip install torchaudio==0.9.0
!pip install transformers

     |████████████████████████████████| 1.9MB 7.2MB/s 
     |████████████████████████████████| 2.5MB 8.2MB/s 
     |████████████████████████████████| 901kB 40.0MB/s 
     |████████████████████████████████| 3.3MB 43.6MB/s 


In [ ]:
!nvidia-smi

Tue Jun 29 17:30:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |   1486MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!kaggle competitions download -c itmo-acoustic-event-detection-2021
!ls 
!cp /content/gdrive/MyDrive/учебка/магистратура/2/ED/audio_train.zip ./ && unzip audio_train.zip 

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/7e87466d3d15d239c76d.wav  
  inflating: train/4f074423bf80bd1d9ec1.wav  
  inflating: train/fb17cc422d31e33c0361.wav  
  inflating: train/1297b7ce3e2bf0b2616b.wav  
  inflating: train/b8dd1e127150f08839ea.wav  
  inflating: train/1827319165c12ed31c49.wav  
  inflating: train/11207f0d9cc9dca98f66.wav  
  inflating: train/e70e28df5f6589f07807.wav  
  inflating: train/6d73d327ce4a29fd3451.wav  
  inflating: train/94c0253d630bdad083b7.wav  
  inflating: train/44ef27554643b4b7f6f8.wav  
  inflating: train/7a7c6d8286ee27edce5d.wav  
  inflating: train/8b8b98d2c4e333d46428.wav  
  inflating: train/85b4139b61ebde4844e7.wav  
  inflating: train/f1152c89cb45bd58571e.wav  
  inflating: train/78e6efb958ba903c9af8.wav  
  inflating: train/9083a29106e08cb37e6b.wav  
  inflating: train/4d879f084c742ec4ded5.wav  
  inflating: train/016f0dd264b6f65736e3.wav  
  inflating: train/a50b4a1226e84fb7b6f8.wav  
  inflating:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import pickle
import random
from multiprocessing import Pool
from functools import partial
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from tqdm.notebook import tqdm
from torchaudio import transforms
#from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
# path to your train/test/meta folders
DATA_PATH = '/content/'
os.makedirs(DATA_PATH, exist_ok=True)

# names of valuable files/folders
train_meta_fname = '/content/gdrive/MyDrive/учебка/магистратура/2/ED/train.csv'
#test_meta_fname = 'sample_submission.csv'
train_data_folder = 'train'
test_data_folder = 'test'

In [ ]:
import transformers

In [ ]:
df_train = pd.read_csv(train_meta_fname)
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [ ]:
n_classes = df_train.label.nunique()
print(n_classes)
label2id = {cl:i for i,cl in enumerate(df_train.label.unique())}
id2label = {v:k for k,v in label2id.items()}
df_train['label_encoded'] = df_train.label.map(label2id)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [ ]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
#from datasets import load_dataset
#import soundfile as sf
processor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")
#w2v = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")

# def map_to_array(batch):
#   speech, _ = sf.read(batch["file"])
#   batch["speech"] = speech
#   return batch

# ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
# ds = ds.map(map_to_array)

# input_values = processor(ds["speech"][0], return_tensors="pt").input_values  # Batch size 1
# hidden_states = model(input_values).last_hidden_state

In [ ]:
class W2WOnlineDataset(torch.utils.data.Dataset):
    def __init__(self, dir, fnames=None, file2label=None, cached=False, norm=True, 
                 wav_len=32000, use_shift=False):
        super().__init__()
        self.sample_rate = 16000
        self.dir = dir
        if fnames is None:
            fnames = os.listdir(self.dir)
        self.fnames = fnames

        self.file2label = file2label
      
        self.norm = norm
        self.wav_len=wav_len
        self.use_shift = use_shift
       
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, index):
        #index, feats_id = self.feats_id2file_feats_id[feats_index]
        fname = self.fnames[index]
        label = None if self.file2label is None else self.file2label[fname]
        wav, sr = torchaudio.load(os.path.join(self.dir, fname))
        if wav.shape[1] < self.wav_len:
          wav = torch.cat([wav, torch.zeros(1, self.wav_len-wav.shape[1])], dim=1)
        if self.use_shift and  wav.shape[1] > self.wav_len:
          shift = random.randint(0, wav.shape[1] - self.wav_len)
        else:
          shift=0
        wav = wav[:, shift:shift + self.wav_len]

        return {
            'index': index,
            'fname': fname,
            'wav': wav.squeeze().numpy(),
            'label': label
        }
        
def collate(b):
    preprocessed = processor([e['wav'] for e in b], return_tensors='pt', sampling_rate=16000)
    return {
        'indices': [e['index'] for e in b],
        'fnames': [e['fname'] for e in b],
        'wavs': preprocessed.input_values,
        'labels': torch.LongTensor([e['label'] for e in b]) if b[0]['label'] is not None else None
    }
        
    
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)
# train_loader = DataLoader(
#                         CachableDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
#                         batch_size=41
#                 )

val_ds = W2WOnlineDataset(os.path.join(DATA_PATH, train_data_folder), 
                         fnames=val_df['fname'].values,
                        file2label={row['fname']:row['label_encoded'] for i,row in val_df.iterrows()},
                        use_shift=False)

train_ds = W2WOnlineDataset(os.path.join(DATA_PATH, train_data_folder), 
                         fnames=train_df['fname'].values,
                        file2label={row['fname']:row['label_encoded'] for i,row in train_df.iterrows()},
                        use_shift=True)



# test_loader = DataLoader(
#                         CachableDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
#                         batch_size=41, shuffle=False
#                 )

In [ ]:
for i in range(len(val_ds)):
  assert val_ds[i]['wav'].shape[0] == 32000, i

In [ ]:
val_ds[2]['wav'].shape

(32000,)

In [ ]:

class Wav2Vec2ED(nn.Module):
    
    def __init__(self,  sample_rate=16000, n_classes=41):
        super().__init__()
        #self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)
        #self.bn0 = nn.BatchNorm2d(1)
        
        #self.cnn1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3, padding=1)
        #self.bn1 = nn.BatchNorm2d(16)
        #self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        #self.bn2 = nn.BatchNorm2d(32)
        #self.cnn3 = nn.Conv2d(in_channels=16, out_channels=3, kernel_size=3, padding=1)
        #self.bn3 = nn.BatchNorm2d(3)
        self.dropout=nn.Dropout(0.1)
        
        self.features = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
        for param in  self.features.encoder.pos_conv_embed.parameters():
          param.requires_grad = False
        # use it as features
#         for param in self.features.parameters():
#             param.requires_grad = False
        self.mp = torch.nn.MaxPool1d(3)
        self.conv1d1 = nn.Conv1d(768, 512, kernel_size=3, stride=3)
        self.conv1d2 = nn.Conv1d(512, 256, kernel_size=5, stride=5)
        self.lin1 = nn.Linear(256*2, 111)
              
        self.lin2 = nn.Linear(111, n_classes)
        self.freeze = True

    def forward(self, x):
        #x = self.ms(x)
        #x = self.bn0(x)
        #x = F.relu(self.cnn1(x))
        #x = self.dropout(F.relu(self.bn1(self.cnn1(x))))
        #x = self.dropout(F.relu(self.bn2(self.cnn2(x))))
        #x = F.relu(self.bn3(self.cnn3(x)))
        if self.freeze:
          with torch.no_grad():
            x = self.features(x).last_hidden_state
        else:
          x = self.features(x).last_hidden_state

#         x = x.view(x.shape[0], -1)
        #print(x.shape)
        x = F.relu(x.transpose(1, 2))
        x = self.mp(x)
        x = F.relu(self.conv1d1(x))
        x = F.relu(self.conv1d2(x))
        #print(x.shape)
        x = x.flatten(1)
        #x = torch.mean(x, dim=1)
        #x = x.flatten(1)
        #x = F.relu(x)
        
        x = F.relu(self.lin1(x))
        #x = F.relu(self.lin2(x))
        x = self.lin2(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x, dim=-1)
        return x

out = Wav2Vec2ED()(torch.ones(1, 32000))
print(out.shape)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 41])


In [ ]:
def eval_model(model, eval_dataset, device='cuda'):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for batch in tqdm(eval_dataset):
            wavs, labs = batch['wavs'].to(device), batch['labels'].numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [ ]:
btz=120 # 32
val_loader = DataLoader(val_ds,
                        batch_size=btz, shuffle=False, collate_fn=collate, num_workers=2)
train_loader = DataLoader(train_ds,
                        batch_size=btz, collate_fn=collate, num_workers=2)
# test_loader = DataLoader(test_ds,
#                         batch_size=btz, collate_fn=collate)


In [ ]:

if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
criterion = nn.CrossEntropyLoss()
model = Wav2Vec2ED()
model = model.to(device)
lr = 1e-4
n_epoch = 50
unfreeze= 25
#model.freeze = False
model_name=f'w2v2Model'
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
#                                     max_lr=lr, 
#                                     total_steps =(n_epoch+1) *len(train_loader)
#                                    )

best_f1 = 0
train_global_loss = []
for epoch in range(n_epoch):
    train_loss=[]
    model.train()
    t= tqdm(train_loader)
    for batch in t:
        optimizer.zero_grad()
        wavs, labs = batch['wavs'].to(device), batch['labels'].to(device)
        #print(wavs.shape)
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        t.set_description(desc="Ep " + str(epoch + 1) + ", train loss " + str(round(loss.item(), 4)))
        #scheduler.step() 
#     if epoch % 10 == 0:
    if epoch == unfreeze:
      print("Unfreeze w2v")
      model.freeze = False
    f1 = eval_model(model, val_loader, device=device)
    #f1_train = eval_model(model, train_loader)
    train_global_loss.append(sum(train_loss)/len(train_loss))
    print(f'epoch: {epoch+1}, f1_test: {f1}, loss_train: {train_global_loss[-1]}. lr: {optimizer.param_groups[0]["lr"]}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), f'../{model_name}.pt')   
    # if (epoch+1) % 30 ==0 :
    #     lr = lr * 0.9 # 0.99
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = lr

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")



epoch: 0, f1_test: 0.015743497694717206, loss_train: 3.6728879809379578. lr: 0.0001



epoch: 1, f1_test: 0.04596918365195451, loss_train: 3.5099943743811712. lr: 0.0001



epoch: 2, f1_test: 0.06834429806828354, loss_train: 3.268686122364468. lr: 0.0001


...


epoch: 45, f1_test: 0.6429975740621863, loss_train: 0.3985811960366037. lr: 0.0001



epoch: 46, f1_test: 0.6309531208513328, loss_train: 0.22794483084645537. lr: 0.0001



epoch: 47, f1_test: 0.6352519438008903, loss_train: 0.2500641348047389. lr: 0.0001



epoch: 48, f1_test: 0.6408424849328711, loss_train: 0.3117421484655804. lr: 0.0001



epoch: 49, f1_test: 0.6318072346856676, loss_train: 0.3440142832696438. lr: 0.0001


In [ ]:
!cp ../w2v2Model.pt '/content/gdrive/MyDrive/учебка/магистратура/2/ED'

In [ ]:
torch.save(model.state_dict(), f'/content/gdrive/MyDrive/учебка/магистратура/2/ED/w2v2Model.last.pt')   

In [ ]:
#w_classes = 1 - np.bincount(np.fromiter(train_ds.file2label.values(), dtype=int))/len(train_ds.file2label)
counts =np.bincount(np.fromiter(train_ds.file2label.values(), dtype=int))
print(counts)
w_classes = 1/counts
print(w_classes)

[ 64 139  69  61 144 149  50 143 138 125 154 147 111 153  97  48 146 144
 125 133 132 126  58 156  84 132 146 143 159  89  79  70 158 114 140 129
  69  58  52  49  63]
[0.015625   0.00719424 0.01449275 0.01639344 0.00694444 0.00671141
 0.02       0.00699301 0.00724638 0.008      0.00649351 0.00680272
 0.00900901 0.00653595 0.01030928 0.02083333 0.00684932 0.00694444
 0.008      0.0075188  0.00757576 0.00793651 0.01724138 0.00641026
 0.01190476 0.00757576 0.00684932 0.00699301 0.00628931 0.01123596
 0.01265823 0.01428571 0.00632911 0.00877193 0.00714286 0.00775194
 0.01449275 0.01724138 0.01923077 0.02040816 0.01587302]


In [ ]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(w_classes).float().to(device))
model = Wav2Vec2ED().to(device)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/учебка/магистратура/2/ED/w2v2Model.last.pt'))
model.freeze = False
# model = Wav2Vec2ED()
# model = model.to(device)
lr = 1e-4
n_epoch = 30
unfreeze= -1
#model.freeze = False
model_name=f'w2v2Model.finetune'
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
#                                     max_lr=lr, 
#                                     total_steps =(n_epoch+1) *len(train_loader)
#                                    )

best_f1 = 0
train_global_loss = []
for epoch in range(n_epoch):
    train_loss=[]
    model.train()
    t= tqdm(train_loader)
    for batch in t:
        optimizer.zero_grad()
        wavs, labs = batch['wavs'].to(device), batch['labels'].to(device)
        #print(wavs.shape)
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        t.set_description(desc="Ep " + str(epoch + 1) + ", train loss " + str(round(loss.item(), 4)))
        #scheduler.step() 
#     if epoch % 10 == 0:
    if epoch == unfreeze:
      print("Unfreeze w2v")
      model.freeze = False
    f1 = eval_model(model, val_loader, device=device)
    #f1_train = eval_model(model, train_loader)
    train_global_loss.append(sum(train_loss)/len(train_loss))
    print(f'epoch: {epoch+1}, f1_test: {f1}, loss_train: {train_global_loss[-1]}. lr: {optimizer.param_groups[0]["lr"]}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), f'/content/gdrive/MyDrive/учебка/магистратура/2/ED/{model_name}.pt')   
    if (epoch+1) % 1 == 0 :
        lr = lr * 0.99 # 0.99
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.codevectors', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)



epoch: 1, f1_test: 0.6276274513910797, loss_train: 1.2532671548818286. lr: 0.0001



epoch: 2, f1_test: 0.6510490815532054, loss_train: 0.9253282625424234. lr: 9.900000000000001e-05



epoch: 3, f1_test: 0.6545483704809363, loss_train: 0.8074660254152197. lr: 9.801e-05


...


epoch: 24, f1_test: 0.6648299761669085, loss_train: 0.4334280098739423. lr: 7.936142836436556e-05



epoch: 25, f1_test: 0.6857334073404859, loss_train: 0.3841505172221284. lr: 7.856781408072191e-05



epoch: 26, f1_test: 0.6989143414812476, loss_train: 0.39115949445649195. lr: 7.778213593991469e-05



epoch: 27, f1_test: 0.6848557950918843, loss_train: 0.3615923610172774. lr: 7.700431458051554e-05



epoch: 28, f1_test: 0.6796025517900355, loss_train: 0.4069150439218471. lr: 7.623427143471038e-05


In [ ]:
!cp ../w2v2Model.fitune.pt '/content/gdrive/MyDrive/учебка/магистратура/2/ED'

0.6398998965159972

In [ ]:
# make a model
# model_name = 'baseline_fulldiv.pt'
# model_name='btz256_fulldiv_lrd0.95'

# model = BaseLineModel().to(device)
# model.load_state_dict(torch.load(os.path.join('..', model_name+".pt")))
model.eval()
print(eval_model(model, val_loader, device=device))
forecast = []
fetacast_fnames = []
with torch.no_grad():
    for batch in tqdm(test_loader):
        feats = batch['feats'].to(device)
        outputs = model.inference(feats)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
        fetacast_fnames.append(batch['fnames'])
labels = [id2label[x] for sublist in forecast for x in sublist]
fnames = [f for sublist in fetacast_fnames for f in sublist]

df_test = pd.DataFrame({"fname":fnames, 'label':labels})
df_test.to_csv(f'{model_name}.csv', index=None)